# Example with ESGPT Synthetic Data

### Set-up

Imports

In [ ]:
from pathlib import Path

import yaml
from bigtree import print_tree
from EventStream.data.dataset_polars import Dataset
from omegaconf import DictConfig

from aces import config, predicates, query

Directories

In [ ]:
config_path = "../../../sample_configs/inhospital-mortality.yaml"
data_path = "../../../sample_data/esgpt_sample"

Configuration File

In [ ]:
# From: https://github.com/caldwell/renderjson

import json
import uuid

from IPython.display import display, display_html, display_javascript


class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript(
            """
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """
            % (self.uuid, self.json_str),
            raw=True,
        )

In [ ]:
with open(config_path, "r") as stream:
    data_loaded = yaml.safe_load(stream)
RenderJSON(data_loaded)

In [ ]:
cfg = config.TaskExtractorConfig.load(config_path=config_path)

Task Tree

In [ ]:
tree = cfg.window_tree
print_tree(tree)

Data

In [ ]:
ESD = Dataset.load(Path(data_path))
events_df = ESD.events_df
dynamic_measurements_df = ESD.dynamic_measurements_df

display(events_df)
display(dynamic_measurements_df)

Predicate Columns

In [ ]:
data_config = DictConfig({"path": data_path, "standard": "esgpt", "ts_format": "%m/%d/%Y %H:%M"})

In [ ]:
predicates_df = predicates.get_predicates_df(cfg=cfg, data_config=data_config)
display(predicates_df)

### End-to-End Query

In [ ]:
df_result = query.query(cfg=cfg, predicates_df=predicates_df)
display(df_result)